In [94]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [95]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [96]:
#Set the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("./")
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('./') 

In [97]:
Pubchem_data=pd.read_csv("./Original data/Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [98]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [99]:
#Load the data
Raw_data= pd.read_csv('./Original data/Smiles_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./Original data/X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Models/Scaler_transformer.pkl')

In [100]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Original data/Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7260 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [84]:
aDesc.set_input_SMILES(Pubchem_data['Canonical_smiles'].tolist())
if not aDesc.calculate_descriptors(X_scaled_data.columns.tolist()):
 print('Error:' + aDesc.get_error())
else:
 X_test=pd.DataFrame(aDesc.get_output())
X_test.columns=X_scaled_data.columns
X_test.index=Pubchem_data.index
X_test=X_test.dropna(axis=0,how='any')
print_RxC(X_test)
X_test.head()

7257 Moleculars × 144 Descriptors


,MATS3p,MATS3v,SM10_AEA(dm),Eig01_EA(ed),SpMax_EA(ed),GATS3v,GATS7s,F07[N-O],SM12_EA(ed),SM09_EA(ed),...,minaaN,nLevel6,s2_pathLength,SpMAD_B(p),SpMAD_A,SM15_EA(ri),GNar,SM03_EA(ed),X0A,GATS8i
cid,,,,,,,,,,,,,,,,,,,,,
5273,-0.118664,-0.058679,11.188448,11.188448,11.188448,1.019144,1.426707,0.0,29.020239,21.785780,...,3.928533,0.000000,2.000000,1.535243,1.332879,16.409915,2.186042,7.110696,0.683381,0.429598
13463,-0.110019,-0.104518,11.607041,11.607041,11.607041,1.012946,0.945439,2.0,29.609609,22.312589,...,3.829388,1.500000,1.500000,1.490869,1.296386,16.935488,2.050213,7.548029,0.714134,0.961675
39148,-0.147753,-0.148145,11.852001,11.852001,11.852001,1.144038,0.840440,4.0,29.936267,22.584873,...,3.879882,1.500000,2.000000,1.463070,1.272769,17.200267,2.017548,7.689371,0.721233,0.743204
56328,-0.030999,-0.050012,11.427769,11.427769,11.427769,0.997521,1.053703,2.0,29.330533,22.061832,...,3.813682,1.333333,1.333333,1.504888,1.304852,16.634942,2.086115,7.379632,0.705461,0.899745
67358,-0.094554,-0.067810,11.970594,11.970594,11.970594,1.023463,1.121252,4.0,29.981979,22.597242,...,3.712999,2.000000,1.500000,1.473359,1.278174,17.229851,2.018429,7.667158,0.721940,0.902616


In [106]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./Original data/X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7257 Moleculars × 144 Descriptors


,MATS3p,MATS3v,SM10_AEA(dm),Eig01_EA(ed),SpMax_EA(ed),GATS3v,GATS7s,F07[N-O],SM12_EA(ed),SM09_EA(ed),...,minaaN,nLevel6,s2_pathLength,SpMAD_B(p),SpMAD_A,SM15_EA(ri),GNar,SM03_EA(ed),X0A,GATS8i
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.148099,0.399835,0.279629,0.279629,0.279629,0.678756,0.937287,-0.166667,0.319368,0.325022,...,0.948180,-0.181818,0.272727,0.922290,1.023979,0.317587,1.040594,0.219248,-0.022413,-0.234805
13463,0.167609,0.224229,0.382490,0.382490,0.382490,0.660760,0.371194,0.166667,0.464641,0.497602,...,0.924251,0.363636,0.090909,0.667298,0.722310,0.517450,0.585611,0.552864,0.442403,0.557069
39148,0.082452,0.057093,0.442684,0.442684,0.442684,1.041388,0.247689,0.500000,0.545159,0.586801,...,0.936438,0.363636,0.272727,0.507550,0.527074,0.618139,0.476195,0.660686,0.549701,0.231925
56328,0.345941,0.433041,0.338437,0.338437,0.338437,0.615972,0.498540,0.166667,0.395852,0.415455,...,0.920460,0.303030,0.030303,0.747859,0.792291,0.403159,0.705872,0.424404,0.311320,0.464901
67358,0.202511,0.364857,0.471826,0.471826,0.471826,0.691296,0.577994,0.500000,0.556426,0.590853,...,0.896160,0.545455,0.090909,0.566675,0.571759,0.629389,0.479147,0.643741,0.560377,0.469174
